In [ ]:
import tifffile
from csbdeep.utils import normalize

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# input_name = r"D:/git/our_data/demo/20210929_171808C.bin"
input_name = r"D:/git/our_data/20210325_144131C.bin"
print_process = False
debug_print = False
debug_matlab_print = True
print(input_name)
output_name = input_name.replace(".bin","_normalize_test.tif")
# output_name = r"D:/git/our_data/20210308_095750C_processed.tif"
print(output_name)

In [ ]:
test_count = 1
def write_output(pic, name = None):
    global test_count
    if "int" in str(pic.dtype) :
        print("in write_output do normalize")
        from csbdeep.utils import normalize
        pic = normalize(pic, 1,99.8, axis= (0,1,2) )
    global output_name
    if name == None :
        save_name = output_name.replace(".tif","_"+str(test_count)+".tif")
        test_count += 1
    else :
        save_name = output_name.replace(".tif","_"+str(name)+".tif")
    print("save name :", save_name)
    tifffile.imsave(save_name, pic)

In [ ]:
if "bin" in input_name :
    f = open(input_name, "rb")   # Trying to open the bin file
    if "processed" in input_name :
        data_pic = np.fromfile(f, dtype=np.float32, count=-1, offset=0)
    else :
        data_pic = np.fromfile(f, dtype=np.int32, count=-1, offset=0)
    
    print("do reshape")
    data_pic = data_pic.reshape((-1,1024,1024))   # 看起來是不用 (-1,1024,1024,1)
    
    

In [ ]:
# 好像一開始就要轉 float
data_pic = data_pic.astype("float32")

In [ ]:
if debug_print :
    # 確認
    print(data_pic.shape)
    print(data_pic.dtype)
    plt.imshow(data_pic[100], cmap='gray')
    print(data_pic[100])

In [ ]:
# tag 0 XY 前處理 (高斯模糊)
from scipy.ndimage import gaussian_filter
sigma=1
Image_Stack_Blur = np.array([
    gaussian_filter(data_pic[i], sigma=sigma, mode = 'nearest', truncate = np.ceil(2*sigma)/sigma )  # imgaussfilt
        for i in range(data_pic.shape[0])])

# 我是覺得 3D 一起做效果應該會比較好
# Image_Stack_Blur = gaussian_filter(data_pic, sigma=1,mode = 'nearest',truncate=2.0)

In [ ]:
if debug_print :
    # 確認
    print(Image_Stack_Blur.shape)
    print(Image_Stack_Blur.dtype)
    plt.imshow(Image_Stack_Blur[100], cmap='gray')
    print(Image_Stack_Blur[100])

    if print_process :
        write_output(Image_Stack_Blur, name = "Image_Stack_Blur")

In [ ]:
# print(Image_Stack_Blur[:,:,300])
# plt.imshow(Image_Stack_Blur[:,:,300], cmap='gray')

In [ ]:
# tag 1 XY 前處理 (參數)
Adj1_A=1
Adj2_A=1
Buffer_Length=10
Cmin_Buffer_Array=np.zeros(Buffer_Length)
Cmax_Buffer_Array=np.zeros(Buffer_Length)

In [ ]:
Image_Stack_Blur_AHist_Corr = np.zeros( shape = data_pic.shape , dtype=data_pic.dtype ) 
# tag 2 XY 前處理 (根據前10張的亮度 進行亮度調整)
# 會先把數值調整到 接近0~1 之間 ((Current_XY_Image - Cmin_Now)/(Cmax_Now - Cmin_Now))
# 再計算前10張的亮度的數值 反向還原 (Image_Temp*(Cmax_Buffered-Cmin_Buffered)+Cmin_Buffered)

# 會有分界問題 ?? 
    # 分界會變得比較不明顯

# Cmin_Buffered 應該要用原圖的數值 還是 調整後的數值 ?? (不過影響應該不大)

for p in range(data_pic.shape[0]):
# for p in range(1):
    Current_XY_Image = Image_Stack_Blur[p]
    # print(Current_XY_Image.shape)
    ROI_Image = Current_XY_Image[49:-50,49:-50]  # index 問題
    # print(ROI_Image.shape)
    # 變成一維才能sort
    ROI_Image = ROI_Image.flatten()
    # print("len ROI_Image : ",len(ROI_Image))
    
    Sort_ascend = np.sort(ROI_Image)
    # print("Sort_ascend :",Sort_ascend)
    # Sort_descend = np.sort(ROI_Image)
    # print("Sort_ascend min :", Sort_ascend[0])
    # print("Sort_ascend max :", Sort_ascend[-1])

    get_place = round(len(Sort_ascend)*Adj1_A/100.0)-1  # index 問題
    

    Cmin_Now = Sort_ascend[get_place]
    Cmax_Now = Sort_ascend[-get_place + 1]  # 是不是這裡出問題
    # if p in [98,99,100,101,102] :
    #     print("now p : ",p)
    #     print("Cmin_Now :", Cmin_Now)
    #     print("Cmax_Now :", Cmax_Now)
    #     print("get_place :",get_place)

    Cmin_Buffer_Array[p%Buffer_Length] = Cmin_Now
    Cmax_Buffer_Array[p%Buffer_Length] = Cmax_Now

    Cmin_Buffered = np.mean(Cmin_Buffer_Array[np.nonzero(Cmin_Buffer_Array)])
    Cmax_Buffered = np.mean(Cmax_Buffer_Array[np.nonzero(Cmax_Buffer_Array)])

    Image_Temp=(Current_XY_Image - Cmin_Now)/(Cmax_Now - Cmin_Now) 
    # Image_Temp 在接近 0~1 之間(會小於0 大於1)
    Image_Adjed = Image_Temp*(Cmax_Buffered-Cmin_Buffered)+Cmin_Buffered
    # 前10張的亮度的平均數值 使用上面那一行相反的運算方式還原回去
    Image_Adjed = np.clip(Image_Adjed, a_min = 0, a_max=None)

    Image_Stack_Blur_AHist_Corr[p] = Image_Adjed


In [ ]:
if debug_matlab_print :
    print("Image_Stack_Blur_AHist_Corr W: " + str(Image_Stack_Blur_AHist_Corr[49,243,347]))
    print("Image_Stack_Blur_AHist_Corr G: " + str(Image_Stack_Blur_AHist_Corr[49,79,146]) )
    print("Image_Stack_Blur_AHist_Corr B: " + str(Image_Stack_Blur_AHist_Corr[49,50,210]) )

In [ ]:
if debug_print :
    # 確認
    print(Image_Stack_Blur_AHist_Corr.shape)
    print(Image_Stack_Blur_AHist_Corr.dtype)
    plt.imshow(Image_Stack_Blur_AHist_Corr[100], cmap='gray')
    print(Image_Stack_Blur_AHist_Corr[100])

    if print_process :
        write_output(Image_Stack_Blur_AHist_Corr)

In [ ]:
if debug_print :
    print(Image_Stack_Blur_AHist_Corr[0,0,300])
    iii = Image_Stack_Blur_AHist_Corr[:,:,300]
    print(iii)
    plt.imshow(iii, cmap='gray')


In [ ]:
# tag 4.0 XZ 前處理(參數) 
Median_1=1
Median_2=1
Blur_1=5
Blur_2=20
Exp_1=1
Exp_2=1.5
Exp_3=0.45

C_max_Def=2 # 3.9  %4.5
C_min_Def=0.6 # 2.1  %2.7

Nperc=0.03
Cmin_Ratio=0.48

Cmax_AB=0.99
Cmin_AB=0.42

In [ ]:
# 這裡開始對 XZ plane 做處理
from math import log

Processed_B_Stack = np.zeros( shape = data_pic.shape , dtype=data_pic.dtype ) 
# tag 3 XZ 前處理
for p in range(Image_Stack_Blur_AHist_Corr.shape[2]):
# for p in range(1):
    Image_XZ_Original = Image_Stack_Blur_AHist_Corr[:,p,:]
    Image_Log = np.log(Image_XZ_Original)
    Image_Log = np.where( Image_Log == -np.inf, np.nan, Image_Log ) # 因為 log(0) 會是負無限大 所以要把這些值去除
    Image_Log_flat = Image_Log.flatten()
    Image_Log_Sort_1D = np.sort(Image_Log_flat)
    Image_Log_Sort_1D_front = Image_Log_Sort_1D[0:round(Image_Log_Sort_1D.shape[0]/10.0)-1] # index 問題
    Min_by_Sort = np.nanmean(Image_Log_Sort_1D_front)
    # print("Min_by_Sort :",Min_by_Sort) # 沒問題
    Image_Log = Image_Log - Min_by_Sort
    Image_Log = np.where( Image_Log == np.nan, 0, Image_Log)
    Image_Log = np.clip(Image_Log, a_min = 0, a_max=None)

    # tag 4 XZ  log 完後 模糊處理
    Image_Blur_1 = gaussian_filter(Image_Log, sigma = Blur_1, mode = 'nearest', truncate = np.ceil(2*Blur_1)/Blur_1)
    # print("Image_Blur_1 :",Image_Blur_1)
    Image_Blur_2 = gaussian_filter(Image_Log, sigma = Blur_2, mode = 'nearest', truncate = np.ceil(2*Blur_2)/Blur_2)
    # print("Image_Blur_2 :",Image_Blur_2)

    from scipy.signal import medfilt2d
    # 中值濾波 

    Image_1st_Backup = Image_Log-Image_Blur_1
    # Image_1st_Backup = medfilt2d(Image_1st_Backup,(Median_1, Median_1))
    Image_1st = Image_1st_Backup**Exp_1
    Image_1st = (Image_1st/np.max(Image_1st))*np.max(Image_1st_Backup)

    Image_2nd_Backup = Image_Blur_1 / Image_Blur_2
    # Image_2nd_Backup = medfilt2d(Image_2nd_Backup,(Median_2, Median_2))
    Image_2nd = Image_2nd_Backup**Exp_2
    Image_2nd = (Image_2nd/np.max(Image_2nd))*np.max(Image_2nd_Backup)

    Image_3rd_Backup = Image_Blur_2
    Image_3rd = Image_3rd_Backup**Exp_3
    Image_3rd = (Image_3rd / np.max(Image_3rd)) * np.max(Image_3rd_Backup)

    # print("Image_1st")
    # print(Image_1st)
    # print("Image_2nd")
    # print(Image_2nd)
    # print("Image_3rd")
    # print(Image_3rd)
    # 這裡都是 Log 之後的結果在做運算 所以變成很小的數字也合情合理 (反正下一步 是會拓展成0~1之間)
    Image_Rec = Image_3rd * Image_2nd + Image_1st    #.*Image_Map_1st;

    Processed_B_Stack[:,p,:]=Image_Rec

In [ ]:
if debug_matlab_print :
    print("Processed_B_Stack W: " + str(Processed_B_Stack[49,243,347]))
    print("Processed_B_Stack G: " + str(Processed_B_Stack[49,79,146]) )
    print("Processed_B_Stack B: " + str(Processed_B_Stack[49,50,210]) )

In [ ]:
if debug_print :
    plt.imshow(Image_Rec, cmap='gray')
    print(Image_Rec)

In [ ]:
if debug_print :
    print("Image_1st")
    print(Image_1st)
    print("Image_2nd")
    print(Image_2nd)
    print("Image_3rd")
    print(Image_3rd)

In [ ]:
if debug_print :
    # 確認
    print(Processed_B_Stack.shape)
    print(Processed_B_Stack.dtype)
    plt.imshow(Processed_B_Stack[100], cmap='gray')
    print(Processed_B_Stack[100])

    if print_process :
        write_output(Processed_B_Stack,"Processed_B_Stack")

In [ ]:
if debug_print :
    print(np.max(Processed_B_Stack))
    print(np.min(Processed_B_Stack))

In [ ]:
if debug_print :
    plt.imshow(Processed_B_Stack[100], cmap='gray')
    print(Processed_B_Stack)

In [ ]:
# tag 5 最後處理

# 把數值調整到 0~1 之間 (超過的數值會調整)
Adj1=1; 
Adj2=1; #0.3

In [ ]:
Histogram_Corr_Image_Stack = Processed_B_Stack.copy()
# 還是對 XZ, 做正規化 
for p in range(Processed_B_Stack.shape[1]):
    Current_Image = Processed_B_Stack[:,p,:]
    ROI_Image=Current_Image[29:,99:925] # index 問題
    if p == 99 :
        print("ROI_Image final value :",ROI_Image[99][99])

    ROI_Image = ROI_Image.flatten()
    Sort_ascend = np.sort(ROI_Image)
    if p == 99 :
        print("Sort_ascend len :", len(Sort_ascend))

    get_place = round(len(Sort_ascend)*Adj1/100.0)-1 # index 問題
    Cmin = Sort_ascend[get_place]
    Cmax = Sort_ascend[ -get_place + 1 ]*1.15

    Image_Norm=(Current_Image-Cmin) / (Cmax-Cmin)
    Image_Norm = np.clip(Image_Norm, a_min = 0, a_max=1)
    Histogram_Corr_Image_Stack[:,p,:] = Image_Norm

In [ ]:
if debug_matlab_print :
    print("Histogram_Corr_Image_Stack W: " + str(Histogram_Corr_Image_Stack[49,243,347]))
    print("Histogram_Corr_Image_Stack G: " + str(Histogram_Corr_Image_Stack[49,79,146]) )
    print("Histogram_Corr_Image_Stack B: " + str(Histogram_Corr_Image_Stack[49,50,210]) )

In [ ]:
# 看一下最終結果
plt.imshow(Histogram_Corr_Image_Stack[100], cmap='gray')

In [ ]:
write_output(Histogram_Corr_Image_Stack,"Normalize")